In [2]:
pip install keras-tcn

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.10.0.dev20220617 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split
from tcn import compiled_tcn

# 데이터 로드

In [3]:
curr_dir = os.getcwd()

forders_tr = os.listdir(curr_dir+"\\data")

data = pd.read_csv(os.path.join(curr_dir+"\\data\\")+forders_tr[0])
data = data.drop(data.columns[0], axis=1)
data

submission = pd.read_csv('2-1_검증데이터셋.csv')
submission1 = submission.copy()          

# 전처리

In [121]:
soil = pd.read_csv('OBS_AAOS_TIM_20220802210224.csv', encoding='cp949')       # 외부 데이터 로드
stn = list(set(soil['지점'].values))

soil_df1 = []      # 일시 포함된 데이터 프레임
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    soil_df1.append(df)

    
soil_df = []        # 일시가 제외된 데이터 프레임 (추합을 위해)
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df = df.drop(['지점', '지점명', '일시'], axis=1)   
    soil_df.append(df) 
  
day = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')    # 21.11.26~22.03.20
day
day1 = pd.DataFrame(day)
day1.columns = ['일자']
day1
day1.to_csv('day_real.csv', index=False)


for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df.to_csv(os.path.join(str(stn[i])+'.csv'), index=False)   # 관측소마다.csv 파일로 저장

In [251]:
# 관측소마다 일시(일자) 통합을 위해 'day_real'파일과 위에서 저장된 모든 관측소에 대해 엑셀 VLOOKUP 함수 이용하여 
# 중간에 비어 있는 일시를 채워주는 작업 후, 각 파일을 _1.csv 파일로 저장

soil_df2 = []
for i in range(len(stn)):
    df = pd.read_csv(os.path.join(str(stn[i])+'_1.csv'), encoding='cp949')
    df = df.drop('일자', axis=1)
    df = df.fillna(0)
    soil_df2.append(df)



soil_df3 = soil_df2[0].copy()
for i in range(len(soil_df[0].columns)):
    soil_df2[0][soil_df2[0].columns[i]] = 0

# 각 관측소마다 평균 내기 (추합)
for i in range(1, len(soil_df2)):
    soil_df3 = soil_df3 + soil_df2[i]
soil_df3 = soil_df3/len(soil_df2)
soil_df3['일자'] = pd.date_range('2021-11-26', periods=95*24+20*24, freq='1H')


# 제공 데이터와의 통합을 위해 제공 데이터에 없는 일시를 외부 데이터에서 제외시킴
d1 = soil_df3[soil_df3['일자'] == '2021-12-22 0:00'].index[0]
for i in range(d1, d1+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)

d2 = soil_df3[soil_df3['일자'] == '2022-03-01 0:00'].index[0]
for i in range(d2, d2+24*3):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d3 = soil_df3[soil_df3['일자'] == '2022-03-14 0:00'].index[0]
for i in range(d3, d3+24):
    soil_df3 = soil_df3.drop(i,axis=0)
    
d4 = soil_df3[soil_df3['일자'] == '2022-03-16 0:00'].index[0]
for i in range(d4, d4+24*2):
    soil_df3 = soil_df3.drop(i,axis=0)

soil_df3 = soil_df3.reset_index()
soil_df3 = soil_df3.drop('index', axis=1)

# 제공 데이터와 외부 데이터 통합 (시간 데이터 제외 모든 열)
col = ['smart_farm.insolation', 'smart_farm.out_tmperature',\
       'smart_farm.out_humidity', 'smart_farm.wind_speed',\
       'smart_farm.wind_direction', 'smart_farm.in_tmperature',\
       'smart_farm.in_humidity', 'smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']          

data_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(data)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(data[col[k]][60*i:60*(i+1)]))
    data_[col[k]] = h
    
h1 = []
for i in range(int(len(data)/60)):
    h1.append(np.sum(data['smart_farm.heat_supply'][60*i:60*(i+1)]))
data_['smart_farm.heat_supply'] = h1



real_data = pd.concat([soil_df3, data_], axis=1)
real_data1 = pd.concat([real_data[real_data.columns[4]], real_data[real_data.columns[:4]], real_data[real_data.columns[5:]]], axis=1)
real_data1 = real_data1.reset_index()
real_data1 = real_data1.drop('index', axis=1)
real_data1 = real_data1.dropna(subset = ['일자'])
real_data1.to_csv('real_data_통합.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.86it/s]


In [4]:
# 위에서 저장한 'real_data_통합' 파일과 'day_real' 파일 이용하여
# 엑셀 VLOOKUP 함수 이용하여 21.11.26부터 22.03.20까지 비어 일자 채운 후, 다시 'real_data_결측' 파일로 저장

pre = pd.read_csv('real_data_결측.csv', encoding='cp949')
pre1 = pre.interpolate(method ='linear')      
pre1 = pre1.dropna(subset = ['일자'])
pre1.to_csv('real_data_최종.csv', index=False)  # 최종 전처리 파일(결측치 제거한)

real1 = pd.read_csv('real_data_최종.csv', encoding='utf-8')
real1

# 자른 이유? 20개를 추출하기 위해 일자 슬라이싱이 필요함.
training = real1[:-504]
testing = real1[-504:]
testing = testing.reset_index()
testing = testing.drop('index', axis=1)

# 모델 1

In [5]:
X = training.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(training_data.iloc[i,-1])
    
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]):
  x_test.append(test_data[i-24:i])
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)


x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)


model11 = compiled_tcn(return_sequences=False,
                      num_feat=x_test.shape[2],
                      nb_filters=50,    
                      num_classes=0,
                      kernel_size=20,     
                      dilations=[2 ** i for i in range(9)],
                      nb_stacks=1,
                      max_len=x_test.shape[1],
                      use_skip_connections=True,
                      regression=True,
                      dropout_rate=0.5,     
                      output_len=1)  

model11.fit(x_train, y_train, batch_size=256, epochs=10)   

(1555, 24, 17)
(1555,)
(653, 24, 17)
(653,)
(480, 24, 17)
x.shape= (None, 50)
model.x = (None, 24, 17)
model.y = (None, 1)


C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
7/7 [==============================] - 31s 4s/step - loss: 110592.7344
Epoch 2/10
7/7 [==============================] - 23s 3s/step - loss: 918.4780
Epoch 3/10
7/7 [==============================] - 25s 4s/step - loss: 142.3555
Epoch 4/10
7/7 [==============================] - 26s 4s/step - loss: 47.8482
Epoch 5/10
7/7 [==============================] - 26s 4s/step - loss: 27.0091
Epoch 6/10
7/7 [==============================] - 29s 4s/step - loss: 18.0243
Epoch 7/10
7/7 [==============================] - 30s 4s/step - loss: 14.4829
Epoch 8/10
7/7 [==============================] - 33s 5s/step - loss: 14.0250
Epoch 9/10
7/7 [==============================] - 31s 4s/step - loss: 11.4577
Epoch 10/10
7/7 [==============================] - 30s 4s/step - loss: 9.3002


In [6]:
p = model11.predict(x_validation)
p = p.flatten()

h1 = []
for i in range(int(len(p)/24)): 
    h1.append(np.sum(p[24*i:24*(i+1)]))    
    
    
h1.insert(0,0)     # 한 번만 클릭!
h11 = h1[4:14] + h1[18:21]
       
submission1['heat_supply_day1'] = h11
submission1 

15/15 [==============================] - 4s 221ms/step


,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,351.349426,NaN
1,2022,3,5,359.693665,NaN
2,2022,3,6,355.478973,NaN
3,2022,3,7,348.627045,NaN
4,2022,3,8,376.912476,NaN
5,2022,3,9,381.386292,NaN
6,2022,3,10,374.886505,NaN
7,2022,3,11,365.554932,NaN
8,2022,3,12,376.823059,NaN
9,2022,3,13,400.294617,NaN


# 모델 2(절감 모델)

In [7]:
X = training.drop(['일자','smart_farm.heat_supply'], axis=1)
y = training[['smart_farm.heat_supply']].values

training_data, test_data, y_tr, y_te = train_test_split(X, y, test_size=0.3, shuffle=False)  

validation = testing.drop(['일자','smart_farm.heat_supply'], axis=1)


x_train = []
y_train = []

for i in range(24, training_data.shape[0]):
  x_train.append(training_data[i-24:i])
  y_train.append(training_data.iloc[i,-1])
    
x_test = []
y_test = []

for i in range(24, test_data.shape[0]): 
  x_test.append(test_data[i-24:i])
  y_test.append(test_data.iloc[i,-1])
x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)



x_validation = []
for i in range(24, validation.shape[0]):
  x_validation.append(validation[i-24:i])
x_validation = np.array(x_validation)


print(x_train.shape) 
print(y_train.shape) 
print(x_test.shape)
print(y_test.shape)
print(x_validation.shape)

model11 = compiled_tcn(return_sequences=False,
                      num_feat=x_test.shape[2],
                      nb_filters=50,    
                      num_classes=0,
                      kernel_size=20,     
                      dilations=[2 ** i for i in range(9)],
                      nb_stacks=1,
                      max_len=x_test.shape[1],
                      use_skip_connections=True,
                      regression=True,
                      dropout_rate=0.5,     
                      output_len=1)  

model11.fit(x_train, y_train, batch_size=256, epochs=10)   

(1555, 24, 21)
(1555,)
(653, 24, 21)
(653,)
(480, 24, 21)
x.shape= (None, 50)
model.x = (None, 24, 21)
model.y = (None, 1)
Epoch 1/10


C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 34s 4s/step - loss: 82398.1797
Epoch 2/10
7/7 [==============================] - 25s 4s/step - loss: 1242.5253
Epoch 3/10
7/7 [==============================] - 26s 4s/step - loss: 198.5568
Epoch 4/10
7/7 [==============================] - 30s 4s/step - loss: 74.1797
Epoch 5/10
7/7 [==============================] - 34s 5s/step - loss: 37.5026
Epoch 6/10
7/7 [==============================] - 31s 4s/step - loss: 22.6531
Epoch 7/10
7/7 [==============================] - 30s 4s/step - loss: 20.0195
Epoch 8/10
7/7 [==============================] - 31s 4s/step - loss: 15.2415
Epoch 9/10
7/7 [==============================] - 26s 4s/step - loss: 13.0710
Epoch 10/10
7/7 [==============================] - 28s 4s/step - loss: 11.2389


In [8]:
p2 = model11.predict(x_validation)  
p2 = p2.flatten()

h2 = []
for i in range(int(len(p2)/24)): 
    h2.append(np.sum(p2[24*i:24*(i+1)]))
h2.insert(0,0)     # 한 번만 클릭!
h22 = h2[4:14] + h2[18:21]

submission1['heat_supply_day2'] = h22
submission1   

15/15 [==============================] - 4s 195ms/step


,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,351.349426,314.582092
1,2022,3,5,359.693665,339.583008
2,2022,3,6,355.478973,332.223877
3,2022,3,7,348.627045,340.040405
4,2022,3,8,376.912476,333.282349
5,2022,3,9,381.386292,329.134369
6,2022,3,10,374.886505,336.489594
7,2022,3,11,365.554932,345.151917
8,2022,3,12,376.823059,359.525574
9,2022,3,13,400.294617,353.903412


In [12]:
submission1.to_csv('220224.csv', index=False) 